In [1]:
import os, glob
import numpy as np
import pandas as pd
import math
import pickle
import matplotlib.pyplot as plt
#from scipy.stats import norm
# from scipy import stats
# from statsmodels.stats.weightstats import ztest
from IPython.display import display, HTML

In [2]:
non_coding_region= "donor"
cancer_type= "Lung"

In [3]:
base_dir = "/home/pdutta/Data/Cancer_wiseGDC/New_data/{}/Generated_files/DNABERT_Data/{}/Patient_wise".format(cancer_type, non_coding_region)
output_dir= "/home/pdutta/Data/Cancer_wiseGDC/New_data/{}/Generated_files/Variant_Analysis/{}/".format(cancer_type, non_coding_region)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [4]:
df_acceptor = pd.read_csv("/home/pdutta/Data/Noncoding_region/Unique_{}_RAW_Combined_BED.tsv".format(non_coding_region), sep="\t")
df_acceptor

,chr_name,start,end,strand,transcript_id,exon_id,transcript_type
0,chr1,12189,12268,+,"ENST00000450305.2, ENST00000456328.2","ENSE00001671638.2, ENSE00002234944.1","transcribed_unprocessed_pseudogene, processed_..."
1,chr1,12659,12738,+,ENST00000450305.2,ENSE00001758273.2,transcribed_unprocessed_pseudogene
2,chr1,12683,12762,+,ENST00000456328.2,ENSE00003582793.1,processed_transcript
3,chr1,13336,13415,+,ENST00000450305.2,ENSE00001746346.2,transcribed_unprocessed_pseudogene
4,chr1,15756,15835,-,ENST00000488147.1,ENSE00002030414.1,unprocessed_pseudogene
...,...,...,...,...,...,...,...
297951,chrY,57211531,57211610,+,ENST00000464205.6_PAR_Y,ENSE00003785071.1,processed_transcript
297952,chrY,57211582,57211661,+,"ENST00000340131.12_PAR_Y, ENST00000359512.8_PA...","ENSE00003627301.1, ENSE00003905275.1, ENSE0000...","retained_intron, protein_coding"
297953,chrY,57213164,57213243,-,ENST00000507418.6_PAR_Y,ENSE00002036959.1,unprocessed_pseudogene
297954,chrY,57213840,57213919,-,ENST00000507418.6_PAR_Y,ENSE00002046926.1,unprocessed_pseudogene


In [5]:
with open("/home/pdutta/Data/Cancer_wiseGDC/New_data/{}/Generated_files/Intersected_Data/intersected_vcf_{}_data.pkl".format(cancer_type, non_coding_region), "rb") as file:
    loaded_dictionary = pickle.load(file)

In [6]:
len(loaded_dictionary)

824

In [7]:
def log_oods_ratio(p1,p2):
    term1 = math.log2(p1/(1-p1))
    term2 = math.log2(p2/(1-p2))
    return (term1-term2)
def calculate_diff_probab(p1,p2):
    return((p2-p1)*max(p1,p2))

In [8]:
patient_dfs= []
patient_id = []
for dirpath, dirnames, filenames in os.walk(base_dir):
    print(dirpath, dirnames, filenames)
    for dir_name in dirnames:
        dir_path = dirpath +"/" + dir_name
        print(dir_path)
        patient_id.append(dir_name)
        print(dir_name)
        #input()
        df_temp = loaded_dictionary[dir_name]
        print(len(df_temp))
        df_temp = df_temp.drop_duplicates(subset=['chr_name', 'start', 'end', 'START_POS', 'END_POS', 'REF','ALT']).reset_index()
        concat_probab = np.load(dir_path+"/Prediction_result/pred_results.npy")
        print(len(concat_probab))
        print(len(df_temp))
        if(2*len(df_temp)==len(concat_probab)):
            #input()
            Reference_probab = concat_probab[::2]
            Alternative_probab = concat_probab[1::2]
            df_temp['Ref_probab']=Reference_probab
            df_temp['Alt_probab']=Alternative_probab
            df_temp['ScoreChange'] = df_temp.apply(lambda row: calculate_diff_probab(row['Ref_probab'], row['Alt_probab']), axis=1)
            df_temp['LogOddRatio'] = df_temp.apply(lambda row: log_oods_ratio(row['Ref_probab'], row['Alt_probab']), axis=1)
            #display(df_temp)
            #input()
            # Apply the filter conditions
            df_temp = df_temp[((df_temp['Ref_probab'] > 0.5) & (df_temp['Alt_probab'] <= 0.5)) | 
                      ((df_temp['Ref_probab'] <= 0.5) & (df_temp['Alt_probab'] > 0.5))]
            #display(df_temp)
            #input()
            patient_dfs.append(df_temp)
        else:
            print(dir_name)
            input()
        #input()
    break

/home/pdutta/Data/Cancer_wiseGDC/New_data/Lung/Generated_files/DNABERT_Data/donor/Patient_wise ['c59b0462-855a-420e-a997-7c75f31c16ed_CaVEMan', '7d0e7267-2303-428d-b42f-4b8cf1d69332_sanger_raw_pindel', 'c59b0462-855a-420e-a997-7c75f31c16ed_sanger_raw_pindel', '514bbe99-1ce0-4c09-8931-d41b44758a86_CaVEMan', '1766972a-0ea3-4230-9d94-db6bbad0a77a_CaVEMan', '92113811-d6b8-436b-b491-ffe17ff2ae10_sanger_raw_pindel', 'bc2d1b67-edac-433e-bf33-ff7958c5dd35_CaVEMan', '58e6ffe3-67ac-4ed8-8c56-c8040b429264_sanger_raw_pindel', '8fb54b3e-8929-4e00-b946-8f87a082162f_CaVEMan', '5a588951-444c-45ce-a3e7-466965ebff1d_sanger_raw_pindel', 'cc121407-bbbd-496c-989c-d98b1092e4ab_sanger_raw_pindel', 'db925a9e-0eaa-4a55-aa99-dca18ff4f236_CaVEMan', 'a0e302b2-ce1a-42e4-9b55-55c7a2cbba45_CaVEMan', 'aea01681-bede-417d-a615-0a6a7124fa6c_CaVEMan', '9dfb5a7b-4f2c-4fb5-a364-8307b22a8f16_sanger_raw_pindel', '5b49038f-da3f-4ba7-9fbb-e1da16ba81b3_CaVEMan', 'aafe897d-6901-4686-958f-e2f6050c2d25_sanger_raw_pindel', 'b30d39e

In [9]:
patient_dfs[0]

,index,chr_name,start,end,strand,transcript_id,exon_id,transcript_type,CHROM,START_POS,...,MP,SG,SNP,SP,TG,TP,Ref_probab,Alt_probab,ScoreChange,LogOddRatio
34,34,chr12,48222022,48222101,+,"ENST00000647575.1, ENST00000654485.1",ENSE00003836383.1,lncRNA,chr12,48222069,...,0.84,CC/CCCTT,False,2.600000e-01,CC/CCCCT,0.57,0.973820,0.298215,-0.657917,6.451772
37,37,chr12,110582769,110582848,-,ENST00000354300.5,ENSE00001512714.3,protein_coding,chr12,110582811,...,0.96,GG/AAGGG,False,2.400000e-01,GG/AGGGG,0.71,0.446544,0.836406,0.326083,-2.663754
59,59,chr17,81123154,81123233,-,"ENST00000326724.9, ENST00000374792.6, ENST0000...","ENSE00003558802.1, ENSE00003478579.1","protein_coding, nonsense_mediated_decay, proce...",chr17,81123168,...,1.00,CC/AACCC,False,6.200000e-02,CC/ACCCC,0.93,0.668466,0.336350,-0.222008,1.992155
77,77,chr19,54698459,54698538,+,ENST00000444199.1,ENSE00002529717.1,unprocessed_pseudogene,chr19,54698460,...,1.00,TT/AATTT,False,1.200000e-01,TT/ATTTT,0.87,0.489163,0.516883,0.014328,-0.160012
95,95,chr20,44066069,44066148,+,"ENST00000341197.9, ENST00000358131.5, ENST0000...","ENSE00003703421.1, ENSE00003706404.1","protein_coding, processed_transcript",chr20,44066099,...,1.00,GG/GGTTT,False,2.900000e-01,GG/GGGTT,0.58,0.218266,0.915572,0.638434,-5.279470
124,124,chr5,1799731,1799810,-,ENST00000508987.1,ENSE00002056706.1,protein_coding,chr5,1799803,...,0.86,TT/CCTTT,False,2.500000e-01,TT/CTTTT,0.61,0.469966,0.508976,0.019855,-0.225335
129,129,chr5,1799770,1799849,-,ENST00000382647.7,ENSE00001492895.5,protein_coding,chr5,1799797,...,0.84,GG/AGGGG,False,3.500000e-01,GG/AAGGG,0.47,0.609321,0.445016,-0.100114,0.959807
131,131,chr5,1799770,1799849,-,ENST00000382647.7,ENSE00001492895.5,protein_coding,chr5,1799806,...,1.00,CC/CCCTT,False,4.000000e-01,CC/CCCCT,0.59,0.609321,0.482276,-0.077411,0.743539
148,148,chr7,63903552,63903631,+,ENST00000656125.1,ENSE00003874819.1,lncRNA,chr7,63903594,...,1.00,CC/AACCC,False,4.100000e-01,CC/ACCCC,0.58,0.326761,0.924264,0.552251,-4.652147
178,178,chr9,62801493,62801572,+,"ENST00000424345.5, ENST00000427509.5","ENSE00001677098.1, ENSE00001681857.1",lncRNA,chr9,62801499,...,0.89,GG/GGGGG,False,1.000000e-01,GG/AGGGG,0.89,0.613585,0.459463,-0.094567,0.901559


In [10]:
# Concatenate the patient dataframes and drop duplicates to get all unique regions across the patients
unique_patient_regions = pd.concat(patient_dfs).drop_duplicates(subset=['chr_name', 'transcript_id', 'start', 'end', 'START_POS', 'END_POS', 'REF', 'ALT'])
unique_patient_regions

,index,chr_name,start,end,strand,transcript_id,exon_id,transcript_type,CHROM,START_POS,...,LogOddRatio,FF017,LEN,OLD_VARIANT,PC,RE,REP,RS,S1,S2
34,34,chr12,48222022,48222101,+,"ENST00000647575.1, ENST00000654485.1",ENSE00003836383.1,lncRNA,chr12,48222069,...,6.451772,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,37,chr12,110582769,110582848,-,ENST00000354300.5,ENSE00001512714.3,protein_coding,chr12,110582811,...,-2.663754,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,59,chr17,81123154,81123233,-,"ENST00000326724.9, ENST00000374792.6, ENST0000...","ENSE00003558802.1, ENSE00003478579.1","protein_coding, nonsense_mediated_decay, proce...",chr17,81123168,...,1.992155,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77,77,chr19,54698459,54698538,+,ENST00000444199.1,ENSE00002529717.1,unprocessed_pseudogene,chr19,54698460,...,-0.160012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,95,chr20,44066069,44066148,+,"ENST00000341197.9, ENST00000358131.5, ENST0000...","ENSE00003703421.1, ENSE00003706404.1","protein_coding, processed_transcript",chr20,44066099,...,-5.279470,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,76,chr12,15956995,15957074,+,"ENST00000428559.7, ENST00000524480.5, ENST0000...","ENSE00003513705.1, ENSE00003529160.1","protein_coding, nonsense_mediated_decay",chr12,15957039,...,4.830038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
379,379,chr8,103819591,103819670,+,"ENST00000262231.14, ENST00000408894.6, ENST000...","ENSE00002191590.1, ENSE00002021482.1, ENSE0000...",protein_coding,chr8,103819604,...,-7.011226,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,15,chr1,218434176,218434255,+,"ENST00000366929.4, ENST00000366930.9",ENSE00001000141.1,protein_coding,chr1,218434243,...,2.706837,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
175,175,chr4,154781894,154781973,+,ENST00000512640.1,ENSE00002056140.1,protein_coding,chr4,154781967,...,0.479222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
filtered_regions = unique_patient_regions.groupby(['chr_name', 'transcript_id', 'start', 'end']).filter(lambda x: len(x) > 1 and not all(x['START_POS'].eq(x['START_POS'].iloc[0]) & x['END_POS'].eq(x['END_POS'].iloc[0]) & x['REF'].eq(x['REF'].iloc[0]) & x['ALT'].eq(x['ALT'].iloc[0])))
filtered_regions

,index,chr_name,start,end,strand,transcript_id,exon_id,transcript_type,CHROM,START_POS,...,LogOddRatio,FF017,LEN,OLD_VARIANT,PC,RE,REP,RS,S1,S2
34,34,chr12,48222022,48222101,+,"ENST00000647575.1, ENST00000654485.1",ENSE00003836383.1,lncRNA,chr12,48222069,...,6.451772,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77,77,chr19,54698459,54698538,+,ENST00000444199.1,ENSE00002529717.1,unprocessed_pseudogene,chr19,54698460,...,-0.160012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
124,124,chr5,1799731,1799810,-,ENST00000508987.1,ENSE00002056706.1,protein_coding,chr5,1799803,...,-0.225335,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
129,129,chr5,1799770,1799849,-,ENST00000382647.7,ENSE00001492895.5,protein_coding,chr5,1799797,...,0.959807,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
131,131,chr5,1799770,1799849,-,ENST00000382647.7,ENSE00001492895.5,protein_coding,chr5,1799806,...,0.743539,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,580,chr11,55648512,55648591,+,ENST00000641692.1,ENSE00003813302.1,protein_coding,chr11,55648528,...,7.368960,True,6.0,.,D,55648539.0,1.0,55648529.0,10.0,418.0060119628906
377,569,chr11,12294819,12294898,+,"ENST00000643523.1, ENST00000646065.1","ENSE00003829278.1, ENSE00001481800.1","retained_intron, protein_coding",chr11,12294834,...,-11.610005,False,3.0,.,I,12294838.0,1.0,12294835.0,6.0,222.05299377441406
687,928,chr14,18333858,18333937,+,ENST00000553170.1,ENSE00002080504.2,processed_transcript,chr14,18333868,...,3.281500,True,1.0,.,D,18333873.0,3.0,18333869.0,32.0,840.4739990234375
1986,2660,chr5,29148874,29148953,+,ENST00000661619.1,ENSE00003851699.1,lncRNA,chr5,29148894,...,4.023185,True,1.0,.,D,29148900.0,4.0,29148895.0,8.0,334.67999267578125


In [12]:
pd.set_option('display.max_columns', None)
filtered_regions[filtered_regions['start']==16052373]

,index,chr_name,start,end,strand,transcript_id,exon_id,transcript_type,CHROM,START_POS,END_POS,ID,REF,ALT,QUAL,FILTER,ASMD,ASRD,CLPM,DP,DS,GP,MP,SG,SNP,SP,TG,TP,Ref_probab,Alt_probab,ScoreChange,LogOddRatio,FF017,LEN,OLD_VARIANT,PC,RE,REP,RS,S1,S2
20,26,chr1,16052373,16052452,+,"ENST00000375667.7, ENST00000375679.9, ENST0000...","ENSE00001638879.1, ENSE00001639784.1, ENSE0000...","protein_coding, nonsense_mediated_decay, retai...",chr1,16052445,16052450,2ef8a3a6-98cd-11ea-979d-bec2ccc58dab,GTCAC,TCAT,240.0,FF010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.575705,0.382741,-0.111090,1.129770,True,5.0,"('chr1:16052445:GGTCAC/GTCAT',)",DI,16052451.0,0.0,16052445.0,9.0,.
19,30,chr1,16052373,16052452,+,"ENST00000375667.7, ENST00000375679.9, ENST0000...","ENSE00001638879.1, ENSE00001639784.1, ENSE0000...","protein_coding, nonsense_mediated_decay, retai...",chr1,16052440,16052442,47b44908-9707-11ea-8993-c154d7ba826d,AG,A,180.0,FF010,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.575705,0.423549,-0.087597,0.884929,True,1.0,.,D,16052447.0,5.0,16052441.0,6.0,254.60499572753906


In [13]:
unique_patient_regions[unique_patient_regions['start']==1043198]

,index,chr_name,start,end,strand,transcript_id,exon_id,transcript_type,CHROM,START_POS,END_POS,ID,REF,ALT,QUAL,FILTER,ASMD,ASRD,CLPM,DP,DS,GP,MP,SG,SNP,SP,TG,TP,Ref_probab,Alt_probab,ScoreChange,LogOddRatio,FF017,LEN,OLD_VARIANT,PC,RE,REP,RS,S1,S2


In [14]:
# Step 3: Create a new dataframe with three rows for each patient, with columns as the acceptor coordinates
# Initialize the dictionaries
data_ref, data_alt, data_log_odd, data_score_change = {}, {}, {}, {}

# Loop over each region in the unique_patient_regions
for _, region in unique_patient_regions.iterrows():
    key = f'{region.chr_name}_{region.start}_{region.end}_{region.START_POS}_{region.END_POS}_{region.REF}_{region.ALT}'
    data_ref[key] = []
    data_alt[key] = []
    data_log_odd[key] = []
    data_score_change[key] = []

# Loop through each patient and for each region, get the Alt_probab score for that region for that patient
for df in patient_dfs:
    print(df.shape)
    #print(df)
    for _, region in unique_patient_regions.iterrows():
        #print(region)
        #input()
        for data, score_column in zip([data_ref, data_alt, data_log_odd, data_score_change], ['Ref_probab', 'Alt_probab', 'LogOddRatio', 'ScoreChange']):
            score_series = df.loc[(df['chr_name'] == region.chr_name) & (df['start'] == region.start)& (df['end'] == region.end) & (df['START_POS'] == region.START_POS) & (df['END_POS'] == region.END_POS) & (df['REF'] == region.REF) & (df['ALT'] == region.ALT) , score_column]
            #print(score_series)
            if score_series.empty:
                score = np.nan
            else:
                score = score_series.max()
                #print(score_series , score)
                #input()
            data[f'{region.chr_name}_{region.start}_{region.end}_{region.START_POS}_{region.END_POS}_{region.REF}_{region.ALT}'].append(score)

(11, 32)
(187, 29)
(184, 29)
(61, 32)
(13, 32)
(184, 29)
(11, 32)
(177, 29)
(11, 32)
(186, 29)
(192, 29)
(7, 32)
(10, 32)
(59, 32)
(169, 29)
(7, 32)
(185, 29)
(14, 32)
(186, 29)
(173, 29)
(3, 32)
(190, 29)
(165, 29)
(3, 32)
(186, 29)
(3, 32)
(11, 32)
(7, 32)
(212, 29)
(192, 29)
(3, 32)
(202, 29)
(170, 29)
(186, 29)
(98, 32)
(53, 32)
(216, 29)
(177, 29)
(198, 29)
(202, 29)
(181, 29)
(12, 32)
(4, 32)
(209, 29)
(186, 29)
(174, 29)
(171, 29)
(182, 29)
(5, 32)
(44, 32)
(8, 32)
(176, 29)
(7, 32)
(195, 29)
(13, 32)
(220, 29)
(174, 29)
(8, 32)
(216, 29)
(182, 29)
(80, 32)
(8, 32)
(11, 32)
(180, 29)
(34, 32)
(5, 32)
(52, 32)
(195, 29)
(9, 32)
(38, 32)
(186, 29)
(7, 32)
(7, 32)
(194, 29)
(7, 32)
(189, 29)
(201, 29)
(27, 32)
(174, 29)
(15, 32)
(193, 29)
(183, 29)
(200, 29)
(8, 32)
(209, 29)
(181, 29)
(170, 29)
(180, 29)
(197, 29)
(188, 29)
(9, 32)
(3, 32)
(196, 29)
(7, 32)
(192, 29)
(9, 32)
(177, 29)
(22, 32)
(4, 32)
(5, 32)
(62, 32)
(16, 32)
(185, 29)
(179, 29)
(209, 29)
(5, 32)
(189, 29)
(177, 

In [15]:
# Reset the option to its default (typically 20)
pd.reset_option('display.max_columns')

In [16]:
result_df_ref = pd.DataFrame(data_ref)
result_df_ref.insert(0, "Patient_ID", patient_id)
result_df_ref

,Patient_ID,chr12_48222022_48222101_48222069_48222070_C_T,chr12_110582769_110582848_110582811_110582812_G_A,chr17_81123154_81123233_81123168_81123169_C_A,chr19_54698459_54698538_54698460_54698461_T_A,chr20_44066069_44066148_44066099_44066100_G_T,chr5_1799731_1799810_1799803_1799804_T_C,chr5_1799770_1799849_1799797_1799798_G_A,chr5_1799770_1799849_1799806_1799807_C_T,chr7_63903552_63903631_63903594_63903595_C_A,...,chr9_112836371_112836450_112836389_112836390_G_T,chr3_53347382_53347461_53347391_53347392_G_A,chr3_149970314_149970393_149970355_149970356_C_A,chr6_158086951_158087030_158086975_158086977_AC_A,chr11_72221020_72221099_72221076_72221077_A_C,chr12_15956995_15957074_15957039_15957040_G_A,chr8_103819591_103819670_103819604_103819605_T_G,chr1_218434176_218434255_218434243_218434244_A_C,chr4_154781894_154781973_154781967_154781968_A_T,chr9_128179333_128179412_128179375_128179376_T_G
0,c59b0462-855a-420e-a997-7c75f31c16ed_CaVEMan,0.97382,0.446544,0.668466,0.489163,0.218266,0.469966,0.609321,0.609321,0.326761,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7d0e7267-2303-428d-b42f-4b8cf1d69332_sanger_ra...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,c59b0462-855a-420e-a997-7c75f31c16ed_sanger_ra...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,514bbe99-1ce0-4c09-8931-d41b44758a86_CaVEMan,NaN,NaN,NaN,NaN,NaN,0.469966,NaN,0.609321,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1766972a-0ea3-4230-9d94-db6bbad0a77a_CaVEMan,NaN,NaN,NaN,0.489163,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
819,fabc8ac4-f575-40c3-9c29-f4a633b0fdd9_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
820,1d6d39c4-b0e1-4d49-ac1d-384212229d5c_sanger_ra...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
821,099b611a-4414-44c3-9458-89b01425d7a4_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.889619,NaN,NaN,NaN,NaN,NaN
822,14428874-2d28-48a5-8a7e-eef30da45e47_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.913846,0.179825,NaN,NaN,NaN


In [17]:
result_df_alt = pd.DataFrame(data_alt)
result_df_alt.insert(0, "Patient_ID", patient_id)
result_df_alt

,Patient_ID,chr12_48222022_48222101_48222069_48222070_C_T,chr12_110582769_110582848_110582811_110582812_G_A,chr17_81123154_81123233_81123168_81123169_C_A,chr19_54698459_54698538_54698460_54698461_T_A,chr20_44066069_44066148_44066099_44066100_G_T,chr5_1799731_1799810_1799803_1799804_T_C,chr5_1799770_1799849_1799797_1799798_G_A,chr5_1799770_1799849_1799806_1799807_C_T,chr7_63903552_63903631_63903594_63903595_C_A,...,chr9_112836371_112836450_112836389_112836390_G_T,chr3_53347382_53347461_53347391_53347392_G_A,chr3_149970314_149970393_149970355_149970356_C_A,chr6_158086951_158087030_158086975_158086977_AC_A,chr11_72221020_72221099_72221076_72221077_A_C,chr12_15956995_15957074_15957039_15957040_G_A,chr8_103819591_103819670_103819604_103819605_T_G,chr1_218434176_218434255_218434243_218434244_A_C,chr4_154781894_154781973_154781967_154781968_A_T,chr9_128179333_128179412_128179375_128179376_T_G
0,c59b0462-855a-420e-a997-7c75f31c16ed_CaVEMan,0.298215,0.836406,0.33635,0.516883,0.915572,0.508976,0.445016,0.482276,0.924264,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7d0e7267-2303-428d-b42f-4b8cf1d69332_sanger_ra...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,c59b0462-855a-420e-a997-7c75f31c16ed_sanger_ra...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,514bbe99-1ce0-4c09-8931-d41b44758a86_CaVEMan,NaN,NaN,NaN,NaN,NaN,0.508976,NaN,0.482276,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1766972a-0ea3-4230-9d94-db6bbad0a77a_CaVEMan,NaN,NaN,NaN,0.516883,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
819,fabc8ac4-f575-40c3-9c29-f4a633b0fdd9_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
820,1d6d39c4-b0e1-4d49-ac1d-384212229d5c_sanger_ra...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
821,099b611a-4414-44c3-9458-89b01425d7a4_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.484751,NaN,NaN,NaN,NaN,NaN
822,14428874-2d28-48a5-8a7e-eef30da45e47_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.271623,0.965851,NaN,NaN,NaN


In [18]:
result_df_score_change = pd.DataFrame(data_score_change)
result_df_score_change.insert(0, "Patient_ID", patient_id)
result_df_score_change

,Patient_ID,chr12_48222022_48222101_48222069_48222070_C_T,chr12_110582769_110582848_110582811_110582812_G_A,chr17_81123154_81123233_81123168_81123169_C_A,chr19_54698459_54698538_54698460_54698461_T_A,chr20_44066069_44066148_44066099_44066100_G_T,chr5_1799731_1799810_1799803_1799804_T_C,chr5_1799770_1799849_1799797_1799798_G_A,chr5_1799770_1799849_1799806_1799807_C_T,chr7_63903552_63903631_63903594_63903595_C_A,...,chr9_112836371_112836450_112836389_112836390_G_T,chr3_53347382_53347461_53347391_53347392_G_A,chr3_149970314_149970393_149970355_149970356_C_A,chr6_158086951_158087030_158086975_158086977_AC_A,chr11_72221020_72221099_72221076_72221077_A_C,chr12_15956995_15957074_15957039_15957040_G_A,chr8_103819591_103819670_103819604_103819605_T_G,chr1_218434176_218434255_218434243_218434244_A_C,chr4_154781894_154781973_154781967_154781968_A_T,chr9_128179333_128179412_128179375_128179376_T_G
0,c59b0462-855a-420e-a997-7c75f31c16ed_CaVEMan,-0.657917,0.326083,-0.222008,0.014328,0.638434,0.019855,-0.100114,-0.077411,0.552251,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7d0e7267-2303-428d-b42f-4b8cf1d69332_sanger_ra...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,c59b0462-855a-420e-a997-7c75f31c16ed_sanger_ra...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,514bbe99-1ce0-4c09-8931-d41b44758a86_CaVEMan,NaN,NaN,NaN,NaN,NaN,0.019855,NaN,-0.077411,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1766972a-0ea3-4230-9d94-db6bbad0a77a_CaVEMan,NaN,NaN,NaN,0.014328,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
819,fabc8ac4-f575-40c3-9c29-f4a633b0fdd9_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
820,1d6d39c4-b0e1-4d49-ac1d-384212229d5c_sanger_ra...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
821,099b611a-4414-44c3-9458-89b01425d7a4_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.360178,NaN,NaN,NaN,NaN,NaN
822,14428874-2d28-48a5-8a7e-eef30da45e47_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-0.586893,0.759184,NaN,NaN,NaN


In [19]:
result_df_log_odd = pd.DataFrame(data_log_odd)
result_df_log_odd.insert(0, "Patient_ID", patient_id)
result_df_log_odd

,Patient_ID,chr12_48222022_48222101_48222069_48222070_C_T,chr12_110582769_110582848_110582811_110582812_G_A,chr17_81123154_81123233_81123168_81123169_C_A,chr19_54698459_54698538_54698460_54698461_T_A,chr20_44066069_44066148_44066099_44066100_G_T,chr5_1799731_1799810_1799803_1799804_T_C,chr5_1799770_1799849_1799797_1799798_G_A,chr5_1799770_1799849_1799806_1799807_C_T,chr7_63903552_63903631_63903594_63903595_C_A,...,chr9_112836371_112836450_112836389_112836390_G_T,chr3_53347382_53347461_53347391_53347392_G_A,chr3_149970314_149970393_149970355_149970356_C_A,chr6_158086951_158087030_158086975_158086977_AC_A,chr11_72221020_72221099_72221076_72221077_A_C,chr12_15956995_15957074_15957039_15957040_G_A,chr8_103819591_103819670_103819604_103819605_T_G,chr1_218434176_218434255_218434243_218434244_A_C,chr4_154781894_154781973_154781967_154781968_A_T,chr9_128179333_128179412_128179375_128179376_T_G
0,c59b0462-855a-420e-a997-7c75f31c16ed_CaVEMan,6.451772,-2.663754,1.992155,-0.160012,-5.27947,-0.225335,0.959807,0.743539,-4.652147,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7d0e7267-2303-428d-b42f-4b8cf1d69332_sanger_ra...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,c59b0462-855a-420e-a997-7c75f31c16ed_sanger_ra...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,514bbe99-1ce0-4c09-8931-d41b44758a86_CaVEMan,NaN,NaN,NaN,NaN,NaN,-0.225335,NaN,0.743539,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1766972a-0ea3-4230-9d94-db6bbad0a77a_CaVEMan,NaN,NaN,NaN,-0.160010,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
819,fabc8ac4-f575-40c3-9c29-f4a633b0fdd9_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
820,1d6d39c4-b0e1-4d49-ac1d-384212229d5c_sanger_ra...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
821,099b611a-4414-44c3-9458-89b01425d7a4_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.098725,NaN,NaN,NaN,NaN,NaN
822,14428874-2d28-48a5-8a7e-eef30da45e47_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.830038,-7.011226,NaN,NaN,NaN


In [20]:
result_df_ref.to_csv(output_dir+"df_ref_score.tsv", sep="\t", index=False)
result_df_alt.to_csv(output_dir+"df_alt_score.tsv", sep="\t", index=False)
result_df_log_odd.to_csv(output_dir+"df_log_odd_score.tsv", sep="\t", index=False)
result_df_score_change.to_csv(output_dir+"df_score_change.tsv", sep="\t", index=False)